In [1]:
import pints
import pints.toy as toy
import numpy as np
import matplotlib.pyplot as plt
import GPy

import pints.emulator as emulator
import pints.emulator.utils as emutils
import pints.emulator.plot as emuplt

import copy
import operator

%matplotlib inline

In [2]:
# Create a model
model = pints.toy.FitzhughNagumoModel()

# Run a simulation
real_parameters = [0.1, 0.5, 3]

# Why time matters? How Euler/Runge-Kutta, stiff ode method

times = np.linspace(0, 20, 200)
org_values = model.simulate(real_parameters, times)

# take 5-10% of range as your std for noise
Vs, Rs = org_values.reshape(2, 200)
V_std = (Vs.max() - Vs.min()) * .1
R_std = (Rs.max() - Rs.min()) * .1

# Add noise
noise = [V_std, R_std]
values = org_values + np.random.normal(0, noise, org_values.shape)

# Create an object with links to the model and time series
problem = pints.MultiOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
real_log_likelihood = pints.KnownNoiseLogLikelihood(problem, noise)

In [3]:
bounds = pints.Boundaries(lower = [0, 0, 2], upper = [1, 1, 4])
log_prior = pints.UniformLogPrior(bounds)

In [4]:
input_parameters = log_prior.sample(500)
likelihoods = np.apply_along_axis(real_log_likelihood, 1, input_parameters)
likelihoods[:5]

array([-1161.48272724, -4277.31589934, -2624.456531  , -2610.00693673,
       -1296.64179098])

In [5]:
emu = emulator.GPEmulator(real_log_likelihood, input_parameters, 
                          likelihoods, 
                          normalize_input = True,
                         )

## Gradually trying bigger and bigger kernels

In [95]:
def is_prod_kernel(kernel):
    return type(kernel) == GPy.kern.src.prod.Prod

def is_add_kernel(kernel):
    return type(kernel) == GPy.kern.src.add.Add

def kernel_to_string(kernel, ident = 0):
    if kernel is None:
        return ""
    s = ""
    if is_prod_kernel(kernel) or is_add_kernel(kernel):
        op = "*" if is_prod_kernel(kernel) else "+"
        sub_kernels = []
        for kern in kernel.parameters:
            sub_kernels.append(kernel_to_string(kern, ident = ident + 1))
        s = op + "\n(" + "\n".join(sub_kernels) + "\n)"
    else:
        # get name of kernel without "'>" characters
        name = str(type(kernel)).split(".")[-1]
        name = name[:-2]

        values = ",".join(["{:5f}".format(x) for x in kernel])
        s = name + "(" + values + ")"
    return " " * ident + s



In [6]:
n_parameters = emu.n_parameters()

base_kerns = [GPy.kern.RBF(n_parameters), 
              GPy.kern.RatQuad(n_parameters),
              GPy.kern.Linear(n_parameters),
              GPy.kern.PeriodicExponential(1),
              GPy.kern.PeriodicMatern52(1),
             ]

max_depth = 10
prev_kern = None
#time_limit?
#objective 
optimizer = 'lfgs'

In [ ]:
def score(emu):
    marg_log_likelihood = emu.get_log_marginal_likelihood()
    marg_log_likelihood

In [7]:
#perform initial kernel selection
max_marg_log_likelihood = -1000
max_kern = None

for kern in base_kerns:
    emu.set_parameters(kernel = kern)
    emu.fit(normalizer = True, messages = False)
    marg_log_likelihood = emu.get_log_marginal_likelihood()
    
    if marg_log_likelihood > max_marg_log_likelihood:
        print('here')
        max_marg_log_likelihood = marg_log_likelihood
        max_kern = emu.get_trained_kern()
    print(emu.get_trained_kern())
    print('Marginal log likelihood: ', marg_log_likelihood)

emu.set_parameters(kernel = max_kern)
emu.fit(normalizer = True)
emu.get_gp()

here
  rbf.         |               value  |  constraints  |  priors
  variance     |  0.5543924450635684  |      +ve      |        
  lengthscale  |  0.7609661405415382  |      +ve      |        
Marginal log likelihood:  -131.23912806365405
here
  RatQuad.     |                value  |  constraints  |  priors
  variance     |   2.4457669097111077  |      +ve      |        
  lengthscale  |   0.4907706901634047  |      +ve      |        
  power        |  0.10568774374540395  |      +ve      |        
Marginal log likelihood:  -110.93806084039551
  linear.    |                value  |  constraints  |  priors
  variances  |  0.18129012527803479  |      +ve      |        
Marginal log likelihood:  -527.0927529870244


 /anaconda3/lib/python3.6/site-packages/GPy/core/gp.py:87: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 3. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel


  periodic_exponential.  |              value  |  constraints  |  priors
  variance               |  6.033662452092788  |      +ve      |        
  lengthscale            |  19.50633749440024  |      +ve      |        
  period                 |  6.168488846212168  |      +ve      |        
Marginal log likelihood:  -575.2004516172092
  periodic_Matern52.  |               value  |  constraints  |  priors
  variance            |  22387.888375562114  |      +ve      |        
  lengthscale         |   16.38033657734034  |      +ve      |        
  period              |   5.032300243844247  |      +ve      |        
Marginal log likelihood:  -570.7384833621109
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0003   5.085148e+02   6.274940e+05 
    00s18  0004   1.530997e+02   6.916331e+03 
    00s72  0021   1.109381e+02   6.809699e-07 
Runtime:     00s72
Optimization status: Converged



GP_regression.,value,constraints,priors
RatQuad.variance,2.4390610555134895,+ve,
RatQuad.lengthscale,0.4908754951285556,+ve,
RatQuad.power,0.10601734409391593,+ve,
Gaussian_noise.variance,0.03190094766754916,+ve,


In [8]:
depth = 1
prev_max_kern = max_kern
while depth < max_depth:
    print("-" * 10 + "Depth " + str(depth) + '-'*10)
    
    max_marg_log_likelihood = -1000
    for op in [operator.add, operator.mul]:
        for kern in base_kerns:
            current_kern = op(prev_max_kern, kern)
            emu.set_parameters(kernel = current_kern)
            emu.fit(normalizer = True, messages = False)
            marg_log_likelihood = emu.get_log_marginal_likelihood()
    
            if marg_log_likelihood > max_marg_log_likelihood:
                max_kern = emu.get_trained_kern()
                max_marg_log_likelihood = marg_log_likelihood
            
    prev_max_kern = max_kern
    print(max_kern)
    print(max_marg_log_likelihood)
    
    depth += 1

----------Depth 1----------


 /anaconda3/lib/python3.6/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


  mul.                           |               value  |  constraints  |  priors
  RatQuad.variance               |   4.778426045178949  |      +ve      |        
  RatQuad.lengthscale            |  0.7934536794328351  |      +ve      |        
  RatQuad.power                  |  0.3429140407963769  |      +ve      |        
  periodic_Matern52.variance     |  22387.888731753745  |      +ve      |        
  periodic_Matern52.lengthscale  |  13.892681347289628  |      +ve      |        
  periodic_Matern52.period       |   4.318229068337911  |      +ve      |        
-89.68383872973828
----------Depth 2----------
  sum.                               |               value  |  constraints  |  priors
  mul.RatQuad.variance               |  17784.958846951333  |      +ve      |        
  mul.RatQuad.lengthscale            |   0.914156217342148  |      +ve      |        
  mul.RatQuad.power                  |  1.5976068298105799  |      +ve      |        
  mul.periodic_Matern52.variance   

 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:327: RuntimeWarning:overflow encountered in multiply
 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:328: RuntimeWarning:invalid value encountered in add
 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:40: RuntimeWarning:overflow encountered in cos
 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/periodic.py:40: RuntimeWarning:invalid value encountered in cos
 /anaconda3/lib/python3.6/site-packages/paramz/transformations.py:108: RuntimeWarning:invalid value encountered in greater
 /anaconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:637: RuntimeWarning:invalid value encountered in log1p
 /anaconda3/lib/python3.6/site-packages/paramz/transformations.py:113: RuntimeWarning:invalid value encountered in greater


LinAlgError: not positive definite, even with jitter.

In [68]:
k = prev_max_kern.parameters[1]

In [74]:
",".join(["{:5f}".format(x) for x in k])

'22387.887871,37.234054,9.695646'

In [96]:
print(kernel_to_string(prev_max_kern))

+
(*
(+
(*
(    RatQuad(17782.096495,9.935765,474.262418)
    PeriodicMatern52(22388.436692,8.181279,1.665435)
)
   RatQuad(2352.008435,1042.695107,492965.179776)
)
  PeriodicMatern52(22447.295905,162.686001,7.258628)
  PeriodicMatern52(22447.090302,113.132427,5.851398)
  RBF(313932.692024,3562634.158011)
)
 PeriodicMatern52(22387.887871,37.234054,9.695646)
)
